In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI, AuthenticationError
import psycopg
import re
import pandas as pd

In [ ]:
load_dotenv()

In [ ]:
api_key = os.environ["OPENAI_KEY"]

def create_client(api_key):
    try:
        client = OpenAI(api_key=api_key)
        client.models.list()
        return client
    except AuthenticationError:
        print("Incorrect API")
    return None

client = create_client(api_key)

In [ ]:
def db_engine():
    host = os.environ.get("DB_HOST")
    port = os.environ.get("DB_POST")
    user = os.environ.get("DB_USERNAME")
    password = os.environ.get("DB_PASSWORD")
    db = os.environ.get("DB_DATABASE")
    
    return f"user={user} password={password} host={host} port={port} dbname={db}"

In [ ]:
# db test
try:
    with psycopg.connect(db_engine()) as conn:
        with conn.cursor() as cur:
            cur.execute("SELECT version();")
            version = cur.fetchone()
            print("✅ Wersja PostgreSQL:", version[0])
except Exception as e:
    print("❌ Błąd połączenia z bazą:", e)


In [ ]:
try:
    with psycopg.connect(
       db_engine()
    ) as conn:
        # Utworzenie kursora i wykonanie zapytania
        with conn.cursor() as cur:
            cur.execute("SELECT version();")
            version = cur.fetchone()
            print("Wersja PostgreSQL:", version[0])

except Exception as e:
    print("Błąd połączenia z bazą:", e)

## pierwsza wersja 

In [ ]:
def get_schema_summary():
    """Zwraca opis tabel i kolumn w bazie (jednorazowo, bez wypisywania)"""
    q = """
    SELECT table_name, column_name, data_type
    FROM information_schema.columns
    WHERE table_schema = 'public'
    ORDER BY table_name, ordinal_position;
    """
    with psycopg.connect(db_engine()) as conn, conn.cursor() as cur:
        cur.execute(q)
        rows = cur.fetchall()

    schema = {}
    for table, col, typ in rows:
        schema.setdefault(table, []).append(f"{col} ({typ})")

    text = "Struktura bazy danych:\n"
    for table, cols in schema.items():
        text += f"- {table}: {', '.join(cols)}\n"
    return text


def run_sql_query(sql):
    """Wykonuje zapytanie SELECT i zwraca wynik"""
    if not sql.strip().lower().startswith("select"):
        return "❌ Dozwolone są tylko zapytania SELECT."

    try:
        with psycopg.connect(db_engine()) as conn, conn.cursor() as cur:
            cur.execute(sql)
            rows = cur.fetchall()
            if not rows:
                return "Brak wyników."
            # jeśli wynik to pojedyncza wartość (np. COUNT)
            if len(rows[0]) == 1 and len(rows) == 1:
                return str(rows[0][0])
            # w innym przypadku kilka pierwszych wierszy
            cols = [desc[0] for desc in cur.description]
            results = [dict(zip(cols, r)) for r in rows]
            return results
    except Exception as e:
        return f"Błąd SQL: {e}"


def agent(prompt, context=[]):
    """Asystent SQL — minimalna wersja"""
    # dodaj systemowy kontekst tylko raz
    if not any(msg.get("role") == "system" for msg in context):
        schema_text = get_schema_summary()
        context.insert(0, {
            "role": "system",
            "content": (
                "Jesteś asystentem SQL połączonym bezpośrednio z bazą PostgreSQL. "
                "Masz możliwość samodzielnego wykonywania zapytań SELECT na tej bazie, "
                "a Twój system automatycznie uruchamia każde zapytanie SQL, które podasz. "
                "Dlatego nie pisz, że nie masz dostępu — masz pełen dostęp do odczytu danych. "
                "Używaj zapytań SELECT do uzyskiwania dokładnych odpowiedzi. "
                "Nie wykonuj żadnych zmian (INSERT, UPDATE, DELETE). "
                "Jeśli nie jesteś pewien, o co chodzi w pytaniu, najpierw dopytaj. "
                "Schemat bazy:\n"
                f"{schema_text}"
            )
})

    messages = context + [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(model="gpt-5", messages=messages)
    answer = response.choices[0].message.content.strip()

    # wykrywanie zapytania SQL i automatyczne wykonanie
    match = re.search(r"SELECT\s+.+?;", answer, flags=re.IGNORECASE | re.DOTALL)
    if match:
        sql = match.group(0)
        result = run_sql_query(sql)
        # agent ma teraz odpowiedzieć tylko wynikiem
        return str(result)
    else:
        # jeśli nie ma SQL, to po prostu zwróć odpowiedź tekstową (np. dopytanie)
        return answer


# --- Pętla rozmowy ---
history = []
while True:
    user_input = input("Ty: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    reply = agent(user_input, history)
    print("Agent:", reply)
    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": reply})


## druga werjsa

In [ ]:
def get_schema_summary():
    """Zwraca opis tabel i kolumn w bazie (jednorazowo, bez wypisywania)"""
    q = """
    SELECT table_name, column_name, data_type
    FROM information_schema.columns
    WHERE table_schema = 'public'
    ORDER BY table_name, ordinal_position;
    """
    with psycopg.connect(db_engine()) as conn, conn.cursor() as cur:
        cur.execute(q)
        rows = cur.fetchall()

    schema = {}
    for table, col, typ in rows:
        schema.setdefault(table, []).append(f"{col} ({typ})")

    text = "Struktura bazy danych:\n"
    for table, cols in schema.items():
        text += f"- {table}: {', '.join(cols)}\n"
    return text


def run_sql_query(sql):
    """Wykonuje zapytanie SELECT i zwraca wynik jako DataFrame"""
    sql = sql.strip().rstrip(";")

    if not sql.lower().startswith("select"):
        return "❌ Dozwolone są tylko zapytania SELECT."

    try:
        with psycopg.connect(db_engine()) as conn:
            df = pd.read_sql_query(sql, conn)

        if df.empty:
            return "Brak wyników."

        # Jeśli jedna kolumna i jeden wiersz (np. COUNT)
        if df.shape == (1, 1):
            return str(df.iloc[0, 0])

        return df

    except Exception as e:
        return f"Błąd SQL: {e}"


def extract_sql(text):
    """Wykrywa i wyciąga zapytanie SQL z odpowiedzi modelu"""
    # Blok ```sql ... ```
    m = re.search(r"```(?:sql)?\s*(SELECT[\s\S]+?)\s*```", text, flags=re.IGNORECASE)
    if m:
        return m.group(1).strip()
    # Inline `SELECT ...`
    m = re.search(r"`\s*(SELECT[\s\S]+?)\s*`", text, flags=re.IGNORECASE)
    if m:
        return m.group(1).strip()
    # Zwykły SELECT ...;
    m = re.search(r"(SELECT[\s\S]+?;)", text, flags=re.IGNORECASE)
    if m:
        return m.group(1).strip()
    # SELECT bez średnika
    m = re.search(r"(SELECT[\s\S]+)$", text, flags=re.IGNORECASE)
    if m:
        return m.group(1).strip()
    return None


def agent(prompt, context=[]):
    """Asystent SQL — z automatycznym wykonywaniem SELECT i wynikami w DataFrame"""
    # dodaj systemowy kontekst tylko raz
    if not any(msg.get("role") == "system" for msg in context):
        schema_text = get_schema_summary()
        context.insert(0, {
            "role": "system",
            "content": (
                "Jesteś asystentem SQL połączonym bezpośrednio z bazą PostgreSQL. "
                "Masz możliwość wykonywania zapytań SELECT na tej bazie, "
                "a Twój system automatycznie uruchamia każde zapytanie SQL, które podasz. "
                "Nie pisz, że nie masz dostępu — masz pełen dostęp do odczytu danych. "
                "Używaj zapytań SELECT, by podać dokładne odpowiedzi. "
                "Nie wykonuj żadnych zmian (INSERT, UPDATE, DELETE). "
                "Jeśli nie jesteś pewien, o co chodzi w pytaniu, najpierw dopytaj. "
                "Schemat bazy:\n"
                f"{schema_text}"
            )
        })

    messages = context + [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(model="gpt-4.1", messages=messages)
    answer = response.choices[0].message.content.strip()

    sql = extract_sql(answer)
    if sql:
        result = run_sql_query(sql)
        return result
    else:
        return answer


# --- Pętla rozmowy ---
history = []
while True:
    user_input = input("Ty: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    reply = agent(user_input, history)
    if isinstance(reply, pd.DataFrame):
        print(reply.to_string(index=False))
    else:
        print("Agent:", reply)
    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": str(reply)})